In [ ]:
from MOTorNOT.coils import *
from MOTorNOT.beams import *
from MOTorNOT.mot import *
from MOTorNOT.integration import Solver, generate_initial_conditions

from tqdm import tqdm_notebook as tqdm
import numpy as np
linewidth = 2*np.pi*29e6


# Slowing beam
We study a single beam oriented in the $x$ direction, passing through a set of coils oriented in the $z$ direction:

In [ ]:
beam = UniformBeam(direction=[-1, 0, 0], power=10e-3, radius=5e-3, detuning=-4*linewidth, handedness=1)
coils = QuadrupoleCoils(radius=0.1, offset=0.1, turns=50, current=50, axis=2)

mot = MOT([beam], coils.field)
mot.plot(limits=[(-200e-3, 200e-3), (-10e-3, 10e-3)])

Let's initialize an atom in the beam and track its motion:

In [ ]:
X0 = [[-0.3, 0, 0]]
V0 = [[75, 0, 0]]

sol = Solver(mot.acceleration, X0, V0).run(1e-2, dt=1e-5)
sol.get_particle(0)['x'].plot()

Now let's examine the effectiveness of the slowing for different velocity classes:

In [ ]:
X0, V0 = generate_initial_conditions(-0.3, np.linspace(1, 80, 100), phi=90, theta=90)

sol = Solver(mot.acceleration, X0, V0).run(1e-2, dt=1e-5)
sol.phase_plot()

# 1D MOT
Now let's add a counter-propagating beam to form a 1D MOT at the origin:

In [ ]:
mot.beams.append(UniformBeam(direction=[1, 0, 0], power=10e-3, radius=5e-3, detuning=-4*linewidth, handedness=1))
mot.plot()

Now when we simulate trajectories, atoms within the capture velocity range will be trapped:

In [ ]:
X0, V0 = generate_initial_conditions(-0.3, np.linspace(1, 90, 100), phi=90, theta=90)

sol = Solver(mot.acceleration, X0, V0).run(15e-2, dt=1e-4)
sol.phase_plot()

The capture velocity can be estimated by finding the maximum velocity of the set of atoms within some distance of the phase-space origin at the end of the simulation:

In [ ]:
sol.capture_velocity(rmax=1e-3, vmax=1e-3)

Often we want to optimize capture velocity by varying parameters like the detuning. Let's run a sweep of MOTs with different detunings and analyze the resulting capture velocity:

In [ ]:
def measure_capture_velocity(detuning):
    beam1 = Beam(direction=[-1, 0, 0], power=10e-3, radius=5e-3, detuning=detuning, handedness=1)
    beam2 = Beam(direction=[1, 0, 0], power=10e-3, radius=5e-3, detuning=detuning, handedness=1)
    coils = QuadrupoleCoils(radius=0.1, offset=0.1, turns=50, current=50, axis=2)
    mot = Beams([beam1, beam2], coils.field)
    
    X0, V0 = generate_initial_conditions(-0.3, np.linspace(1, 90, 100), phi=90, theta=90)
    sol = Solver(mot.acceleration, X0, V0).run(5e-2, dt=1e-4)
    
    return sol.capture_velocity(rmax=10e-3, vmax=10e-3)

linewidth = 2*np.pi*atom['gamma']
detunings = np.arange(-4, 0, 0.5)
vc = []

for delta in tqdm(detunings):
    vc = np.append(vc, measure_capture_velocity(delta*linewidth))
    
import matplotlib.pyplot as plt
plt.plot(detunings, vc)

# Six-beam MOT
We can easily construct a 3D MOT using the SixBeamMOT convenience function:

In [ ]:
mot = SixBeam(power=10e-3, radius=5e-3, detuning=-linewidth/2, handedness=1, field=coils.field)
mot.plot()

In realistic experiments, atoms will typically enter between two beams. Let's set up a simulation:

In [ ]:
X0, V0 = generate_initial_conditions(-10e-3, np.linspace(1, 30, 100), phi=45, theta=90)

sol = Solver(mot.acceleration, X0, V0).run(10e-2, dt=1e-5)
sol.phase_plot(trapped_only=True)

Now let's calculate the optimal detuning for this case:

In [ ]:
def measure_capture_velocity(detuning):
    coils = QuadrupoleCoils(radius=0.1, offset=0.1, turns=50, current=50, axis=2)
    mot = SixBeamMOT(power=10e-3, radius=5e-3, detuning=detuning, handedness=1, field=coils.field)
    
    X0, V0 = generate_initial_conditions(-0.3, np.linspace(1, 50, 100), phi=90, theta=90)
    sol = Solver(mot.acceleration, X0, V0).run(100e-2, dt=1e-3)
    
    return sol.capture_velocity()

linewidth = 2*np.pi*atom['gamma']
detunings = np.arange(-4, 0, 0.1)
vc = []

for delta in tqdm(detunings):
    vc = np.append(vc, measure_capture_velocity(delta*linewidth))
    
import matplotlib.pyplot as plt
plt.plot(detunings, vc)

# Grating MOTs
The gratingMOT convenience class lets you construct gMOTs with arbitrary parameters:

In [ ]:
from MOTorNOT.gmot import gratingMOT

gmot = gratingMOT(position = 3e-3, 
                  alpha = 45, 
                  detuning = -linewidth/2, 
                  radius = 8e-3, 
                  power = 20e-3, 
                  handedness = -1, 
                  R1 = 0.33, 
                  field = coils.field)
gmot.plot('xz', limits=[(-15e-3, 15e-3), (-3e-3, 10e-3)], numpoints=80)